In [46]:
import requests
import re
from requests import get
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

print("Modules imported")

Modules imported


In [47]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.implicitly_wait(0)
driver.maximize_window()
print("Driver setup and browser opened")

Driver setup and browser opened


In [48]:
files = open('fptshop_laptop.txt','r')
lines = files.readlines()
list_item = []
for line in lines:
    dict_item = {}
    html = driver.get(line.strip('\n'))
    sleep(3)
    click_detail = driver.find_element(By.CSS_SELECTOR,('#root > main > div > div.l-pd-body > div > div.l-pd-body__wrapper > div.l-pd-body__right > div > div > div'))
    sleep(1)
    click_detail.click()
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    detail = soup.find('div', class_='c-modal__row').text
    detail = detail.replace('Thông tin hàng hóa', '').replace('\xa0', '')
    
    brand_index = detail.find('Thương hiệu')
    origin_index = detail.find('Xuất xứ')
    release_date_index = detail.find('Thời điểm ra mắt')
    warranty_period_index = detail.find('Thời gian bảo hành (tháng)')
    storage_guide_index = detail.find('Hướng dẫn bảo quản')

    # Trích xuất giá trị 'Thương hiệu'
    brand_start = detail.find(':', brand_index) + 1
    brand_end = detail.find('Thương hiệu', brand_start)
    brand = detail[brand_start:brand_end].strip()
    dict_item['Thương hiệu'] = brand

    # Trích xuất giá trị 'Xuất xứ'
    origin_start = detail.find(':', origin_index) + 1
    origin_end = detail.find('Thời điểm ra mắt', origin_start)
    origin = detail[origin_start:origin_end].strip()
    dict_item['Xuất xứ'] = origin

    # Trích xuất giá trị 'Thời điểm ra mắt'
    release_date_start = detail.find(':', release_date_index) + 1
    release_date_end = detail.find('Thời gian bảo hành', release_date_start)
    release_date = detail[release_date_start:release_date_end].strip()
    dict_item['Thời điểm ra mắt'] = release_date

    # Trích xuất giá trị 'Thời gian bảo hành'
    warranty_period_start = detail.find(':', warranty_period_index) + 1
    warranty_period_end = detail.find('Hướng dẫn bảo quản', warranty_period_start)
    warranty_period = detail[warranty_period_start:warranty_period_end].strip()
    dict_item['Thời gian bảo hành'] = warranty_period

    # Trích xuất giá trị 'Hướng dẫn bảo quản'
    storage_guide_start = detail.find(':', storage_guide_index) + 1
    storage_guide_end = detail.find('Hướng dẫn sử dụng', storage_guide_start)
    storage_guide = detail[storage_guide_start:storage_guide_end].strip()
    dict_item['Hướng dẫn bảo hành'] = storage_guide
    
    list_item.append(dict_item)

In [49]:
import pandas as pd
df = pd.DataFrame(list_item)
df

,Thương hiệu,Xuất xứ,Thời điểm ra mắt,Thời gian bảo hành,Hướng dẫn bảo hành
0,Asus,Trung Quốc,4/2023,24,"Để nơi khô ráo, nhẹ tay, dễ vỡ."
1,HP,Trung Quốc,2022,12,"Để nơi khô ráo, nhẹ tay, dễ vỡ."
2,Lenovo,Nội địa - Trung Quốc,05/05/2023,24,"Để nơi khô ráo, nhẹ tay, dễ vỡ."
3,Acer,Trung Quốc,2021,12,"Để nơi khô ráo, nhẹ tay, dễ vỡ."
4,Dell,Trung Quốc,2022,12,"Để nơi khô ráo, nhẹ tay, dễ vỡ."
...,...,...,...,...,...
227,MSI,Trung Quốc,2023,24,"Để nơi khô ráo, nhẹ tay, dễ vỡ."
228,MSI,Trung Quốc,2023,24,"Để nơi khô ráo, nhẹ tay, dễ vỡ."
229,MSI,Trung Quốc,2023,24,"Để nơi khô ráo, nhẹ tay, dễ vỡ."
230,MSI,Trung Quốc,2023,24,"Để nơi khô ráo, nhẹ tay, dễ vỡ."


In [55]:
df['Xuất xứ'] = 'Trung Quốc'
df['Xuất xứ'].value_counts()

Trung Quốc    232
Name: Xuất xứ, dtype: int64

In [57]:
df.to_csv('fpt_laptop_information.csv')